In [78]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

In [109]:
class DGI(nn.Module):
    def __init__(self, x, edge_index, dimensions):
        super(DGI, self).__init__()
        data = Data(x, edge_index)
        self.x = data.x
        self.edge_index = data.edge_index
        self.dim = dimensions
        
        self.h_pos = nn.Parameter(torch.Tensor(data.num_node_features, data.num_nodes))
        self.h_neg = nn.Parameter(torch.Tensor(data.num_node_features, data.num_nodes))
        self.weight = nn.Parameter(torch.Tensor(self.dim, self.dim))
        #nn.Parameter(self.h_pos, self.h_neg, self.summary, self.weight)
        
    def GCNlayer(self):
        self.model = GCNConv(self.x, self.edge_index)
        return nn.PReLU(self.model)
        
    def summary(self, h):
        return torch.mean(h, dim = 0)
    
    def corruption(self):
        return self.x[torch.randperm(self.x.size(0))], self.edge_index
    
    def discriminator(self, h, summary):
        value = torch.matmul(h, torch.matmul(self.weight, summary))
        return torch.sigmoid(value)
    
    def loss(self):
        loss_pos = -torch.log(self.discriminator(self.h_pos, self.summary)).mean()
        loss_neg = -torch.log(1 - self.discriminator(self.h_neg, self,summary)).mean()
        return (loss_pos + loss_neg).mean()
    
    def forward(self):
        h_pos = self.GCNlayer(self.h_pos, self.edge_index)
        x_neg = self.corruption(self.x, self.edge_index)
        h_neg = self.GCNlayer(self.h_neg, self.edge_index)
        summary = self.summary(self.h_pos)
        return h_pos, h_neg, summary     
    
   

In [110]:
def train():
    model.train()
    optimizer.zero_grad()
    h_pos, h_neg, summary = model(data.x, data.edge_index)
    loss = model.loss(h_pos, h_neg, summary)
    loss.backward()
    optimizer.step()
    return loss.item()

In [111]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DGI(data.x, data.edge_index, 512).to(device)
optimizer = torch.optim.Adam(model.parameter(), lr = 0.001)

AttributeError: 'DGI' object has no attribute 'parameter'

In [101]:
print(model.parameters)

<bound method Module.parameters of DGI()>


In [102]:
print(model.parameters)

<bound method Module.parameters of DGI()>


In [6]:
import torch_geometric.utils as util

In [7]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

class GCN(nn.Module):
    def __init__(self, X, A, D, dimension):
        super(self, GCN).__init__()
        self.X = data.x
        self.A = util.to_dense_adj(util.add_self_loops(data.edge_index))
        self.D = util.to_dense_adj(util.degree(util.add_self_loops(data.edge_index)))
        self.dim = dimension
        
    def forward(self):
        weight = torch.rand(self.dim, self.dim)
        @
        
        